<a href="https://colab.research.google.com/github/parulchutaniphd/Face-Recognition/blob/main/Filter_Photographs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install face_recognition

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 9.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566185 sha256=940b74bfb69b22a8dc57f84dfc8ff233f1fbbe7995fd6b0a686baf31938a87a9
  Stored in directory: /root/.cache/pip/wheels/b4/4b/8f/751e99d45f089bdf366a7d3e5066db3c2b84a62e4377f534d7
Successfully built face-recognition-models


In [3]:
import face_recognition
import os
import cv2
from google.colab.patches import cv2_imshow

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
ALLOWED_EXTENSIONS = set(['png', 'jpg', 'jpeg'])

def allowed_file(filename):
  return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

In [19]:
ref_encodings={}

for i in os.listdir('/content/drive/MyDrive/MainFolder/ref images'):
  if i and allowed_file(i):
    ref_name = i.split('.')[0].upper()
    ref_image = face_recognition.load_image_file('/content/drive/MyDrive/MainFolder/ref images/{}'.format(i))
    ref_image_locations = face_recognition.face_locations(ref_image)#, model="cnn")
    ref_image_encoding = face_recognition.face_encodings(ref_image, ref_image_locations)[0]
    ref_encodings[ref_name] = ref_image_encoding

In [23]:
def target_dir(key):
  parent_dir = "/content/drive/MyDrive/MainFolder"
  path = os.path.join(parent_dir, key)
  # print(path)

  if key in os.listdir('/content/drive/MyDrive/MainFolder'):
    # print('Directory already exists')
    pass
  else:
    os.mkdir(path)

  return path

In [24]:
for image in os.listdir('/content/drive/MyDrive/MainFolder'):
   if image and allowed_file(image):
     group_test_image = face_recognition.load_image_file('/content/drive/MyDrive/MainFolder/{}'.format(image))
     group_test_image = cv2.resize(group_test_image, (800,800))
     group_test_image_location = face_recognition.face_locations(group_test_image)
     # group_test_image_encodings = face_recognition.face_encodings(group_test_image)
     
     for i in group_test_image_location:
       (top, right, bottom, left) = i
       cropped_image = group_test_image[top-50:bottom+50, left-50:right+50]
       # cropped_image = group_test_image[top:bottom, left:right]
       cropped_image_locations = face_recognition.face_locations(cropped_image)#, model="cnn")
       cropped_image_encoding = face_recognition.face_encodings(cropped_image)

       if len(cropped_image_encoding)!=0:
         for key in ref_encodings.keys():
           ref_encoding = ref_encodings[key]
           path = target_dir(key)
           faceDis = face_recognition.face_distance(ref_encoding, cropped_image_encoding)
           if faceDis > 0.5:
            # print("No Match")
            pass
           else:
            #  tagged_image = cv2.rectangle(group_test_image, (left, top), (right, bottom), (0, 0, 255), 2)
            #  cv2_imshow(cv2.resize(group_test_image,(500,500)))
             os.chdir(path)
             cv2.imwrite(image, group_test_image)#filename, image
       else:
         pass